In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch

from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import torch.nn as nn
from torch.utils.data import  TensorDataset, DataLoader
import torch.optim as optim


In [3]:
df = pd.read_csv(r"C:\Users\pc\OneDrive\Music\Desktop\ML_Projects\Dataset\DateFruit_Dataset.csv")
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AREA           898 non-null    int64  
 1   PERIMETER      898 non-null    float64
 2   MAJOR_AXIS     898 non-null    float64
 3   MINOR_AXIS     898 non-null    float64
 4   ECCENTRICITY   898 non-null    float64
 5   EQDIASQ        898 non-null    float64
 6   SOLIDITY       898 non-null    float64
 7   CONVEX_AREA    898 non-null    int64  
 8   EXTENT         898 non-null    float64
 9   ASPECT_RATIO   898 non-null    float64
 10  ROUNDNESS      898 non-null    float64
 11  COMPACTNESS    898 non-null    float64
 12  SHAPEFACTOR_1  898 non-null    float64
 13  SHAPEFACTOR_2  898 non-null    float64
 14  SHAPEFACTOR_3  898 non-null    float64
 15  SHAPEFACTOR_4  898 non-null    float64
 16  MeanRR         898 non-null    float64
 17  MeanRG         898 non-null    float64
 18  MeanRB    

In [5]:
# Split X, Y Value

x = df.drop(["Class"], axis=1)
y = df["Class"]

In [6]:
# y Labeling

le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
# Train Test Split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# X Scaling

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [9]:
# Convert data into Tensor Dataset

x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [10]:
# TensorDataset / Dataloader 

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=32)


In [11]:
# ANN Model

class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(x_train.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 7)
        )

    def forward(self, x):
        return self.model(x)

In [12]:
model = ANN()

# Loss & Optim

criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [13]:
# Training the Neural Network

epochs = 100
for epoch in range(epochs):
    model.train()

    running_loss = 0.0

    for xb, yb in train_loader:
        optimizer.zero_grad()

        outputs = model(xb)
        loss = criteria(outputs, yb)
        loss.backward()
        optimizer.step()    # Params Update

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)  

    print(f"epoch = {epoch+1} / {epoch}, loss = {train_loss}")

epoch = 1 / 0, loss = 1.657151180764903
epoch = 2 / 1, loss = 1.054980404998945
epoch = 3 / 2, loss = 0.7026288742604463
epoch = 4 / 3, loss = 0.5195430491281592
epoch = 5 / 4, loss = 0.43303354732368304
epoch = 6 / 5, loss = 0.37876219334809674
epoch = 7 / 6, loss = 0.3383388085209805
epoch = 8 / 7, loss = 0.3093888642995254
epoch = 9 / 8, loss = 0.2889636223730834
epoch = 10 / 9, loss = 0.2575336409651715
epoch = 11 / 10, loss = 0.23157226391460584
epoch = 12 / 11, loss = 0.21935844972081806
epoch = 13 / 12, loss = 0.2056365592972092
epoch = 14 / 13, loss = 0.19538292223992554
epoch = 15 / 14, loss = 0.18277630533861078
epoch = 16 / 15, loss = 0.1725177152649216
epoch = 17 / 16, loss = 0.16728982880063678
epoch = 18 / 17, loss = 0.16035886432813562
epoch = 19 / 18, loss = 0.15025805066461148
epoch = 20 / 19, loss = 0.15027487569529077
epoch = 21 / 20, loss = 0.13705554377773535
epoch = 22 / 21, loss = 0.13197057135403156
epoch = 23 / 22, loss = 0.12767039150323556
epoch = 24 / 23, lo

In [14]:
# Evaluate

model.eval()

total = 0
correct = 0

with torch.no_grad():
    for xb, yb in test_loader:
        outputs = model(xb) # [0.2, 0.5, 1,3, -0.5, ..] - 7 vals
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == yb).sum().item()
        total += yb.size(0) # Actual samples in each batch

print("Accuracy: ", correct/total*100)

Accuracy:  93.88888888888889


In [15]:
# Saving Model

torch.save(model.state_dict(), "best_datefruite_model.pt")

In [17]:
# Loading Model

model.load_state_dict(torch.load("best_datefruite_model.pt"))

<All keys matched successfully>